# Comparing the HIT to GENCODE v47

Assigning HIT transcripts into categories based on their annotation status.

In [1]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pyranges as pr
import matplotlib as mpl
from collections import defaultdict
import numpy as np

/users/jferrer/jmidgley/annotation_project/lib64/python3.9/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
gencode_v47_df = pd.read_csv(
    "../data_raw/gencode.v47.tsv",
    sep="\t",
    dtype=str)
gencode_v47_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,...,exon_id,level,tag,transcript_support_level,havana_transcript,hgnc_id,ont,havana_gene,protein_id,ccdsid
0,chr1,HAVANA,exon,11121,11211,.,+,.,ENSG00000290825.2,ENST00000832824.1,...,ENSE00004248723.1,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,exon,12010,12227,.,+,.,ENSG00000290825.2,ENST00000832824.1,...,ENSE00004248735.1,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,12613,12721,.,+,.,ENSG00000290825.2,ENST00000832824.1,...,ENSE00003582793.1,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,exon,13453,14413,.,+,.,ENSG00000290825.2,ENST00000832824.1,...,ENSE00004248730.1,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,exon,11125,11211,.,+,.,ENSG00000290825.2,ENST00000832825.1,...,ENSE00004248721.1,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155000,chrM,ENSEMBL,exon,14149,14673,.,-,.,ENSG00000198695.2,ENST00000361681.2,...,ENSE00001434974.2,3,appris_principal_1,NaN,NaN,HGNC:7462,NaN,NaN,ENSP00000354665.2,NaN
2155001,chrM,ENSEMBL,exon,14674,14742,.,-,.,ENSG00000210194.1,ENST00000387459.1,...,ENSE00001544476.1,3,Ensembl_canonical,NaN,NaN,HGNC:7479,NaN,NaN,NaN,NaN
2155002,chrM,ENSEMBL,exon,14747,15887,.,+,.,ENSG00000198727.2,ENST00000361789.2,...,ENSE00001436074.2,3,appris_principal_1,NaN,NaN,HGNC:7427,NaN,NaN,ENSP00000354554.2,NaN
2155003,chrM,ENSEMBL,exon,15888,15953,.,+,.,ENSG00000210195.2,ENST00000387460.2,...,ENSE00001544475.2,3,Ensembl_canonical,NaN,NaN,HGNC:7499,NaN,NaN,NaN,NaN


In [3]:
HIT_hg38_df = pd.read_csv(
    "../data_raw/HIT_hg38.tsv",
    sep="\t",
    dtype=str)
HIT_hg38_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,gene_name,gene_type,transcript_classification,protein_id_orf_summary,Intron,Exon,5prime,3prime,closest
0,chr1,PacBio,exon,14360,14829,.,-,.,ENSG00000227232,PBT00037199,WASH7P,pseudogene,Novel Protein coding not overlapping annotated...,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,PacBio,exon,14970,15038,.,-,.,ENSG00000227232,PBT00037199,WASH7P,pseudogene,Novel Protein coding not overlapping annotated...,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,PacBio,exon,15796,15947,.,-,.,ENSG00000227232,PBT00037199,WASH7P,pseudogene,Novel Protein coding not overlapping annotated...,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,PacBio,exon,16607,16765,.,-,.,ENSG00000227232,PBT00037199,WASH7P,pseudogene,Novel Protein coding not overlapping annotated...,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,PacBio,exon,16854,17055,.,-,.,ENSG00000227232,PBT00037199,WASH7P,pseudogene,Novel Protein coding not overlapping annotated...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967844,chrY,StringTie,exon,20582590,20582693,.,+,.,ENSG00000198692,STRT02294670,EIF1AY,protein_coding,Novel Protein coding: internal additional pept...,NaN,NaN,NaN,NaN,NaN,NaN
3967845,chrY,StringTie,exon,20584474,20584524,.,+,.,ENSG00000198692,STRT02294670,EIF1AY,protein_coding,Novel Protein coding: internal additional pept...,NaN,NaN,NaN,NaN,NaN,NaN
3967846,chrY,StringTie,exon,20588024,20588105,.,+,.,ENSG00000198692,STRT02294670,EIF1AY,protein_coding,Novel Protein coding: internal additional pept...,NaN,NaN,NaN,NaN,NaN,NaN
3967847,chrY,StringTie,exon,20589484,20589575,.,+,.,ENSG00000198692,STRT02294670,EIF1AY,protein_coding,Novel Protein coding: internal additional pept...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print("Number unique transcripts")
print("HIT:", len(HIT_hg38_df["transcript_id"].unique()))
print("Number unique genes")
print("HIT:", len(HIT_hg38_df["gene_id"].unique()))

Number unique transcripts
HIT: 376582
Number unique genes
HIT: 22129


In [5]:
# remove duplicated tx (these duplicated transcripts were identified based on GffCompare output from HIT vs GENCODE)
dup_tx = pd.read_csv("../data_processed/duplicated_transcripts.tsv", sep="\t", dtype=str)["transcript_id"].unique()
HIT_hg38_df = HIT_hg38_df[~HIT_hg38_df["transcript_id"].isin(dup_tx)]

In [6]:
print("Number unique transcripts")
print("HIT:", len(HIT_hg38_df["transcript_id"].unique()))
print("Number unique genes")
print("HIT:", len(HIT_hg38_df["gene_id"].unique()))

Number unique transcripts
HIT: 376460
Number unique genes
HIT: 22128


In [7]:
# remove non-standard chr
standard_chromosomes = {f'chr{i}' for i in range(1, 23)}.union({'chrX', 'chrY'})
HIT_hg38_df = HIT_hg38_df[HIT_hg38_df['seqname'].isin(standard_chromosomes)]

In [8]:
print("Number unique transcripts")
print("GENCODE:", len(gencode_v47_df["transcript_id"].unique()))
print("HIT:", len(HIT_hg38_df["transcript_id"].unique()))
print("Number unique genes")
print("GENCODE:", len(gencode_v47_df["gene_id"].unique()))
print("HIT:", len(HIT_hg38_df["gene_id"].unique()))

Number unique transcripts
GENCODE: 385659
HIT: 376343
Number unique genes
GENCODE: 78724
HIT: 22119


In [9]:
# Build exon dictionaries for each chromosome
def build_transcript_dict(df):
    chr_dict = defaultdict(list)
    grouped = df.groupby("transcript_id")
    
    for tid, group in grouped:
        seqname = group["seqname"].iloc[0]
        exon_coords = sorted(list(zip(group["start"].astype(int), group["end"].astype(int))))
        chr_dict[seqname].append((tid, exon_coords))
    return chr_dict

HIT_hg38_df_exons = build_transcript_dict(HIT_hg38_df)
gencode_v47_df_exons = build_transcript_dict(gencode_v47_df)

In [10]:
# Find transcripts fully annotated in GENCODE
def exon_overlap(exons1, exons2, tolerance=100):
    for i, ((s1, e1), (s2, e2)) in enumerate(zip(exons1, exons2)):
        if i == 0:  # first exon
            if abs(s1 - s2) > tolerance or e1 != e2:
                return False
        elif i == len(exons1) - 1:  # last exon
            if abs(e1 - e2) > tolerance or s1 != s2:
                return False
        else:  # internal exon
            if s1 != s2 or e1 != e2:
                return False
    return True


# Find new splice variants with ALL exons annotated (GENCODE transcript may contain additional exons)
def subset_match(HIT_exons, gencode_exons, tolerance=100):
    g_idx = 0
    for i, (hit_start, hit_end) in enumerate(HIT_exons):
        found = False
        while g_idx < len(gencode_exons):
            gen_start, gen_end = gencode_exons[g_idx]
            if i == 0:
                # First HIT exon — allow tolerance only on start
                if abs(hit_start - gen_start) <= tolerance and hit_end == gen_end:
                    found = True
                    g_idx += 1
                    break
            elif i == len(HIT_exons) - 1:
                # Last HIT exon — allow tolerance only on end
                if abs(hit_end - gen_end) <= tolerance and hit_start == gen_start:
                    found = True
                    g_idx += 1
                    break
            else:
                # Internal exon — exact match
                if hit_start == gen_start and hit_end == gen_end:
                    found = True
                    g_idx += 1
                    break

            g_idx += 1
        if not found:
            return False

    return True



def found_exon(HIT_exon, gencode_exon_dict, position, nr_HIT_exons):
    hit_start, hit_end = HIT_exon

    # Always allow an internal exact match
    candidate_keys = [(hit_start, hit_end, 'internal')]

    if position == 0:
        # HIT first exon: allow tolerant 'first' + internal exact
        candidate_keys.append((hit_start, hit_end, 'first'))
    elif position == nr_HIT_exons - 1:
        # HIT last exon: allow tolerant 'last'  + internal exact
        candidate_keys.append((hit_start, hit_end, 'last'))
    else:
        # HIT internal exon: allow exact matches to GENCODE first/last 
        candidate_keys.extend([
            (hit_start, hit_end, 'first'),   
            (hit_start, hit_end, 'last')     
        ])

    matched_tids = set()
    for k in candidate_keys:
        tids = gencode_exon_dict.get(k)
        if tids:
            matched_tids |= tids

    return (len(matched_tids) > 0, matched_tids)

In [11]:
# Build a lookup set of GENCODE exons which includes a tolerance window for the start and end of a tx
def build_tolerant_gencode_set(transcripts, tolerance=100):
    exon_dict = defaultdict(set)
    for tid, exons in transcripts:
        for i, (start, end) in enumerate(exons):
            if i == 0:
                for delta in range(-tolerance, tolerance + 1):
                    exon_dict[(start + delta, end, 'first')].add(tid)
            elif i == len(exons) - 1:
                for delta in range(-tolerance, tolerance + 1):
                    exon_dict[(start, end + delta, 'last')].add(tid)
            else:
                exon_dict[(start, end, 'internal')].add(tid)
    return exon_dict

In [12]:
# Merge exons that are directly adjacent to each other
def merge_adjacent_exons(exons, tid):
    if not exons:
        return []
    
    # Sort exons by start coordinate 
    exons = sorted(exons, key=lambda x: x[0])
    merged = [exons[0]]
    merged_count = 0
    
    for start, end in exons[1:]:
        last_start, last_end = merged[-1]
        
        # If current exon starts directly after the last one ends
        if start == last_end + 1:
            # Merge them
            merged[-1] = (last_start, end)
            merged_count += 1
        else:
            merged.append((start, end))

    return merged

In [13]:
matching_transcripts = {}
subset_transcripts = {}
partially_unannotated_transcripts = {}
fully_unannotated_transcripts = {}
new_combo_annotated_exons = {}

for chr_name in HIT_hg38_df_exons:
    print(chr_name)
    if chr_name not in gencode_v47_df_exons:
        continue
    
    HIT_hg38_transcript_list = HIT_hg38_df_exons[chr_name]
    gencode_v47_transcript_list = gencode_v47_df_exons[chr_name]
    
    # Group GENCODE transcripts by exon count
    gencode_by_exon_count = defaultdict(list)
    for tid, exons in gencode_v47_transcript_list:
        gencode_by_exon_count[len(exons)].append((tid, exons))

    # Build fast exon lookup set (for finding individual annotated exons)
    gencode_exon_set = build_tolerant_gencode_set(gencode_v47_transcript_list)

    
    for HIT_tid, HIT_exons in tqdm(HIT_hg38_transcript_list, desc=f"Transcripts in {chr_name}", leave=False):
        HIT_exons = merge_adjacent_exons(HIT_exons, HIT_tid)
        matched = False
        len_hit = len(HIT_exons)
        
        # Try to match entire transcript exactly
        for gencode_tid, gencode_exons in gencode_by_exon_count[len_hit]:
            if exon_overlap(HIT_exons, gencode_exons):
                matching_transcripts[HIT_tid] = {
                    'chr': chr_name,
                    'exons': HIT_exons,
                    'matched_to': gencode_exons,
                    'matched_ref_id': gencode_tid
                }
                matched = True
                break

        if matched:
            continue
            
        # Try subset match (HIT tx contained in GENCODE tx)
        for longer_len in range(len_hit + 1, max(gencode_by_exon_count.keys()) + 1):
            for gencode_tid, gencode_exons in gencode_by_exon_count[longer_len]:
                # Skip if exon coordinates are out of possible range
                if HIT_exons[0][0] > gencode_exons[-1][1] or HIT_exons[-1][1] < gencode_exons[0][0]:
                    continue
                if subset_match(HIT_exons, gencode_exons):
                    subset_transcripts[HIT_tid] = {
                        'chr': chr_name,
                        'exons': HIT_exons,
                        'matched_to': gencode_exons,
                        'matched_ref_id': gencode_tid
                    }
                    matched = True
                    break
            if matched:
                break
            
        if not matched:
            # If entire tx can't be matched, try to match individual exons
            annotated_info = [
                found_exon(exon, gencode_exon_set, i, len_hit)
                for i, exon in enumerate(HIT_exons)
                ]
            annotated_flags = [flag for flag, _ in annotated_info]
            matched_tids = set().union(*[tids for flag, tids in annotated_info if flag])
     
            # No exons are annotated
            if all(not flag for flag in annotated_flags):
                fully_unannotated_transcripts[HIT_tid] = {
                    'chr': chr_name,
                    'exons': HIT_exons
                }
            # All exons are annotated (new splice variant)
            elif all(annotated_flags):
                ref_match_counts = defaultdict(int)
                for flag, tids in annotated_info:
                    for tid in tids:
                        ref_match_counts[tid] += 1

                # Find the ref ID with the most matches
                if ref_match_counts:
                    # Get the highest match count
                    max_count = max(ref_match_counts.values())
    
                    # Find all ref IDs with that count
                    tied_ref_ids = [tid for tid, count in ref_match_counts.items() if count == max_count]
    
                    if len(tied_ref_ids) == 1:
                        # No tie: take the sole best match
                        best_ref_id = tied_ref_ids[0]
                    else:
                        # Tie: resolve by comparing tx lengths
                        hit_exons = dict(HIT_hg38_transcript_list)[HIT_tid]
                        hit_length = hit_exons[-1][1] - hit_exons[0][0]
        
                        ref_lengths = {}
                        for tid in tied_ref_ids:
                            ref_exons = dict(gencode_v47_transcript_list)[tid]
                            ref_lengths[tid] = ref_exons[-1][1] - ref_exons[0][0]
        
                        # Choose the ref ID with length closest to HIT
                        best_ref_id = min(ref_lengths.items(), key=lambda x: abs(x[1] - hit_length))[0]
                else:
                    best_ref_id = None

                # Save result
                new_combo_annotated_exons[HIT_tid] = {
                    'chr': chr_name,
                    'exons': HIT_exons,
                    'matched_ref_id': list(matched_tids),
                    'best_match_ref_id': best_ref_id
                }
            # Some exons annotated
            elif any(not flag for flag in annotated_flags):
                unannotated_exons = [
                    exon for exon, is_annotated in zip(HIT_exons, annotated_flags) if not is_annotated
                ]
                # Count how many times each ref ID appears across annotated exons
                ref_match_counts = defaultdict(int)
                for flag, tids in annotated_info:
                    if flag:
                        for tid in tids:
                            ref_match_counts[tid] += 1

                # Find the ref ID with the most matches
                if ref_match_counts:
                    # Get the highest match count
                    max_count = max(ref_match_counts.values())
    
                    # Find all ref IDs with that count
                    tied_ref_ids = [tid for tid, count in ref_match_counts.items() if count == max_count]
    
                    if len(tied_ref_ids) == 1:
                        # No tie: take the sole best match
                        best_ref_id = tied_ref_ids[0]
                    else:
                        # Tie: resolve by comparing tx lengths
                        hit_exons = dict(HIT_hg38_transcript_list)[HIT_tid]
                        hit_length = hit_exons[-1][1] - hit_exons[0][0]
        
                        ref_lengths = {}
                        for tid in tied_ref_ids:
                            ref_exons = dict(gencode_v47_transcript_list)[tid]
                            ref_lengths[tid] = ref_exons[-1][1] - ref_exons[0][0]
        
                        # Choose the ref ID with length closest to HIT
                        best_ref_id = min(ref_lengths.items(), key=lambda x: abs(x[1] - hit_length))[0]
                else:
                    best_ref_id = None

                # Save result
                partially_unannotated_transcripts[HIT_tid] = {
                    'chr': chr_name,
                    'unannotated_exons': unannotated_exons,
                    'exons': HIT_exons,
                    'matched_ref_id': list(matched_tids),
                    'best_match_ref_id': best_ref_id
                }

chr1


chr10


chr11


chr12


chr13


...

chr7


chr8


chr9


chrX


chrY


In [14]:
len(matching_transcripts)

38891

In [15]:
len(subset_transcripts)

25891

In [16]:
len(partially_unannotated_transcripts)

257815

In [17]:
len(fully_unannotated_transcripts)

17261

In [18]:
len(new_combo_annotated_exons)

36485

In [19]:
len(matching_transcripts) + len(subset_transcripts) + len(partially_unannotated_transcripts) + len(fully_unannotated_transcripts) + len(new_combo_annotated_exons)

376343

In [20]:
sum(len(v) for v in HIT_hg38_df_exons.values())

376343

### Split up partially unannotated tx into overlapping & non-overlapping

In [21]:
# Convert dictionary of partially unannotated exons into PyRanges
rows = []
for tid, info in partially_unannotated_transcripts.items():
    for exon in info["unannotated_exons"]:
        start, end = exon
        rows.append({
            "Chromosome": info["chr"],
            "Start": start,
            "End": end,
            "transcript_id": tid
        })

partially_unannotated_df = pd.DataFrame(rows)
# Add strand info
strand_info = HIT_hg38_df[['transcript_id', 'strand']].drop_duplicates(subset='transcript_id')
strand_info = strand_info.rename(columns={'strand': 'Strand'})

# Merge 
partially_unannotated_df = partially_unannotated_df.merge(
    strand_info,
    on='transcript_id',
    how='left'
)

partially_unannotated_pr = pr.PyRanges(partially_unannotated_df)


gencode_pr = pr.PyRanges(
    gencode_v47_df.rename(columns={
        "seqname": "Chromosome",
        "start": "Start",
        "end": "End",
        "strand": "Strand"
    })[["Chromosome", "Start", "End", "Strand", "transcript_id"]]
)

In [22]:
# Join the unannotated exon df with gencode exons
overlapping_exon_pu = partially_unannotated_pr.join(gencode_pr)
overlapping_exon_pu = overlapping_exon_pu[overlapping_exon_pu.Strand == overlapping_exon_pu.Strand_b]
overlapping_exon_pu = overlapping_exon_pu.df
overlapping_exon_pu = overlapping_exon_pu.rename(columns={
    "transcript_id": "HIT_transcript_id",
    "transcript_id_b": "gencode_transcript_id"
})

In [23]:
# Assign each exon as overlapping or non-overlapping
overlapping_coords_pu = set(
    tuple(row) for row in overlapping_exon_pu[["Chromosome", "Start", "End"]].values
)

partially_unannotated_df["is_overlapping"] = partially_unannotated_df.apply(
    lambda row: (row["Chromosome"], row["Start"], row["End"]) in overlapping_coords_pu,
    axis=1
)

In [24]:
import pickle

# Save df for downstream analysis of novel exons
with open("../data_processed/partially_unannotated_df.pkl", "wb") as f:
    pickle.dump(partially_unannotated_df, f)

In [25]:
# Split up partially annotated tx into those with exon overlap and those not overlapping gencode exon
non_overlapping_ids_pu = set(
    partially_unannotated_df.loc[partially_unannotated_df["is_overlapping"] == False, "transcript_id"]
)
overlapping_ids_pu = set(partially_unannotated_df["transcript_id"]) - non_overlapping_ids_pu

nonoverlapping_partially_unannotated = {}
overlapping_partially_unannotated = {}

nonoverlapping_partially_unannotated = {
    tid: val for tid, val in partially_unannotated_transcripts.items()
    if tid in non_overlapping_ids_pu
}

overlapping_partially_unannotated = {
    tid: val for tid, val in partially_unannotated_transcripts.items()
    if tid in overlapping_ids_pu
}

### Split up fully unannotated tx into overlapping & non-overlapping

In [26]:
# Now do the same for the fully unannotated tx
rows = []
for tid, info in fully_unannotated_transcripts.items():
    for exon in info["exons"]:
        start, end = exon
        rows.append({
            "Chromosome": info["chr"],
            "Start": start,
            "End": end,
            "transcript_id": tid
        })

fully_unannotated_df = pd.DataFrame(rows)

# Merge with strand info
fully_unannotated_df = fully_unannotated_df.merge(
    strand_info,
    on='transcript_id',
    how='left'
)
fully_unannotated_pr = pr.PyRanges(fully_unannotated_df)


In [27]:
# join the unannotated exon df with gencode exons
overlapping_exon_fu = fully_unannotated_pr.join(gencode_pr)
overlapping_exon_fu = overlapping_exon_fu[overlapping_exon_fu.Strand == overlapping_exon_fu.Strand_b]
overlapping_exon_fu = overlapping_exon_fu.df
overlapping_exon_fu = overlapping_exon_fu.rename(columns={
    "transcript_id": "HIT_transcript_id",
    "transcript_id_b": "gencode_transcript_id"
})

In [28]:
# Compute size of overlap
overlapping_exon_fu["overlap_size"] = (
    overlapping_exon_fu[["Start", "End", "Start_b", "End_b"]]
    .apply(lambda row: max(0, min(row["End"], row["End_b"]) - max(row["Start"], row["Start_b"])), axis=1)
)

In [29]:
# Assign each exon as overlapping or non-overlapping
overlapping_coords_fu = set(
    tuple(row) for row in overlapping_exon_fu[["Chromosome", "Start", "End"]].values
)

fully_unannotated_df["is_overlapping"] = fully_unannotated_df.apply(
    lambda row: (row["Chromosome"], row["Start"], row["End"]) in overlapping_coords_fu,
    axis=1
)

In [30]:
# Compute total overlap size for each HIT-gencode transcript pair
tx_overlap = (
    overlapping_exon_fu
    .groupby(["HIT_transcript_id", "gencode_transcript_id"])["overlap_size"]
    .sum()
    .reset_index()
)

# Select the gencode transcript with the highest total overlap per HIT transcript
best_tx_overlap = (
    tx_overlap
    .sort_values(by=["overlap_size", "gencode_transcript_id"], ascending=[False, True])
    .drop_duplicates(subset=["HIT_transcript_id"])
    .rename(columns={
        "HIT_transcript_id": "transcript_id",
        "gencode_transcript_id": "largest_overlap_gencode_id"
    })[["transcript_id", "largest_overlap_gencode_id"]]
)

# Merge best transcript-level overlap into unannotated_df
fully_unannotated_df = fully_unannotated_df.merge(
    best_tx_overlap,
    on="transcript_id",
    how="left"
)

In [31]:
# Split up fully unannotated tx into those with exon overlap and those not overlapping gencode exon
overlapping_ids_fu = set(
    fully_unannotated_df.loc[fully_unannotated_df["is_overlapping"] == True, "transcript_id"]
)
non_overlapping_ids_fu = set(fully_unannotated_df["transcript_id"]) - overlapping_ids_fu


nonoverlapping_fully_unannotated = {}
overlapping_fully_unannotated = {}

nonoverlapping_fully_unannotated = {
    tid: val for tid, val in fully_unannotated_transcripts.items()
    if tid in non_overlapping_ids_fu
}

overlapping_fully_unannotated = {
    tid: val for tid, val in fully_unannotated_transcripts.items()
    if tid in overlapping_ids_fu
}

for tid in overlapping_fully_unannotated:
    # Get all gencode IDs with the largest overlaps for this transcript
    gencode_ids = fully_unannotated_df.loc[
        fully_unannotated_df["transcript_id"] == tid, "largest_overlap_gencode_id"
    ].dropna().values[0]

    # Add as a new key to the transcript entry
    overlapping_fully_unannotated[tid]["largest_overlap_gencode_id"] = gencode_ids

In [32]:
def dict_to_df(d, label):
    df = pd.DataFrame.from_dict(d, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'transcript_id'}, inplace=True)
    df['category'] = label
    return df

In [33]:
# Convert dictionaries into df
df_nonoverlap_partial = dict_to_df(nonoverlapping_partially_unannotated, "Transcripts with annotated exons and non-overlapping unannotated exons")
df_overlap_partial = dict_to_df(overlapping_partially_unannotated, "Transcripts with annotated exons and only overlapping unannotated exons")
df_unannotated_overlap = dict_to_df(overlapping_fully_unannotated, "Fully unannotated transcripts with overlapping exons")
df_unannotated_nonoverlap = dict_to_df(nonoverlapping_fully_unannotated, "Fully unannotated transcripts without any overlapping exons")
df_combo = dict_to_df(new_combo_annotated_exons, "Unannotated transcripts composed entirely of annotated exons")
df_match = dict_to_df(matching_transcripts, "GENCODE-annotated transcripts")
df_subset = dict_to_df(subset_transcripts, "Unannotated transcripts fully contained within GENCODE transcripts")


combined_df = pd.concat([df_match, df_subset, df_nonoverlap_partial, df_overlap_partial, df_unannotated_overlap, df_unannotated_nonoverlap, df_combo], ignore_index=True)

In [34]:
# Add strand info
combined_df = combined_df.merge(
    strand_info[["transcript_id", "Strand"]],
    how='left',
    on='transcript_id',
)

In [35]:
combined_df["category"].value_counts()

category
Transcripts with annotated exons and only overlapping unannotated exons    193904
Transcripts with annotated exons and non-overlapping unannotated exons      63911
GENCODE-annotated transcripts                                               38891
Unannotated transcripts composed entirely of annotated exons                36485
Unannotated transcripts fully contained within GENCODE transcripts          25891
Fully unannotated transcripts with overlapping exons                        13970
Fully unannotated transcripts without any overlapping exons                  3291
Name: count, dtype: int64

In [36]:
def summarize_tx_category(df):
    tx_category = df['category']
    if tx_category == 'GENCODE-annotated transcripts':
        summarized_category = "GENCODE-annotated transcripts"
        ref_id = df['matched_ref_id']
        
    elif tx_category == "Unannotated transcripts fully contained within GENCODE transcripts":
        summarized_category = "Unannotated transcripts with all exons annotated"
        ref_id = df['matched_ref_id']
        
    elif tx_category == "Unannotated transcripts composed entirely of annotated exons":
        summarized_category = "Unannotated transcripts with all exons annotated"
        ref_id = df['best_match_ref_id']
        
    elif tx_category in ["Transcripts with annotated exons and only overlapping unannotated exons", "Transcripts with annotated exons and non-overlapping unannotated exons"]:
        summarized_category = "Unannotated transcripts with annotated and unannotated exons"
        ref_id = df['best_match_ref_id']
        
    elif tx_category == "Fully unannotated transcripts with overlapping exons":
        summarized_category = "Unannotated transcripts without any annotated exons"
        ref_id = df['largest_overlap_gencode_id']
        
    elif tx_category == "Fully unannotated transcripts without any overlapping exons":
        summarized_category = "Unannotated transcripts without any annotated exons"
        ref_id = 'None'
    
    else:
        summarized_category = 'None'
        ref_id = 'None'
        
    return summarized_category, ref_id

In [37]:
combined_df[['summarized_category', 'ref_id']] = combined_df.apply(summarize_tx_category, axis=1, result_type='expand')

In [38]:
combined_df = combined_df.drop(columns=["matched_ref_id", "best_match_ref_id", "largest_overlap_gencode_id"])
combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1


## Add gene name and type info from Ensembl

In [39]:
gene_names = pd.read_csv('../data_raw/mart_export.txt',
                           sep='\t')
gene_names.head()

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene name,Gene type
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT-TF,Mt_tRNA
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT-RNR1,Mt_rRNA
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT-TV,Mt_tRNA
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT-RNR2,Mt_rRNA
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT-TL1,Mt_tRNA


In [40]:
combined_df['stripped_ref_id'] = combined_df['ref_id'].str.replace(r'\.\d+$', '', regex=True)


combined_df = combined_df.merge(
    gene_names[["Transcript stable ID", "Gene stable ID", "Gene name", "Gene type"]],
    how='left',
    left_on='stripped_ref_id',
    right_on='Transcript stable ID'
).drop(columns="Transcript stable ID")


In [41]:
combined_df["Gene type"].value_counts()

Gene type
protein_coding                        330745
lncRNA                                 40993
transcribed_unprocessed_pseudogene       482
processed_pseudogene                     364
transcribed_unitary_pseudogene           136
TEC                                       59
unprocessed_pseudogene                    56
transcribed_processed_pseudogene          53
IG_V_gene                                 38
snRNA                                     29
misc_RNA                                  16
miRNA                                     15
snoRNA                                    14
TR_C_gene                                  5
scaRNA                                     5
unitary_pseudogene                         5
ribozyme                                   1
Name: count, dtype: int64

In [42]:
combined_df["Gene name"].isna().sum()

np.int64(20239)

In [43]:
combined_df["Gene type"].isna().sum()

np.int64(3327)

In [44]:
combined_df[combined_df['Gene name'].isna()]

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,NaN,lncRNA
121,ENST00000289779.3,chr1,"[(160997957, 160998906), (160999043, 160999091...","[(160997957, 160998906), (160999043, 160999091...",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000289779.7,ENST00000289779,ENSG00000270149,NaN,protein_coding
516,ENST00000362058.2,chr1,"[(16629670, 16631109), (16631530, 16631613), (...","[(16629670, 16631109), (16631530, 16631613), (...",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000362058.2,ENST00000362058,ENSG00000291072,NaN,lncRNA
1482,ENST00000412483.1,chr1,"[(234212606, 234214107), (234214868, 234215088)]","[(234212606, 234214107), (234214868, 234215088)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000412483.1,ENST00000412483,ENSG00000233332,NaN,lncRNA
1504,ENST00000415019.1,chr1,"[(158197922, 158199835), (158203793, 158203877)]","[(158197922, 158199835), (158203793, 158203877)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000415019.1,ENST00000415019,ENSG00000176320,NaN,lncRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376259,STRT02285555,chrX,"[(102599494, 102599847), (102601464, 102601509...",NaN,Unannotated transcripts composed entirely of a...,NaN,+,Unannotated transcripts with all exons annotated,ENST00000466616.6,ENST00000466616,ENSG00000271147,NaN,lncRNA
376300,ENCT00000484963.1,chrY,"[(11161704, 11163137), (11212744, 11212802), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,-,Unannotated transcripts with all exons annotated,ENST00000796795.1,ENST00000796795,ENSG00000291032,NaN,lncRNA
376328,STRT02292412,chrY,"[(12406182, 12406500), (12406796, 12406937), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,-,Unannotated transcripts with all exons annotated,ENST00000737339.1,ENST00000737339,ENSG00000291034,NaN,lncRNA
376330,STRT02293232,chrY,"[(11167047, 11167217), (11212744, 11212802), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,-,Unannotated transcripts with all exons annotated,ENST00000796810.1,ENST00000796810,ENSG00000291032,NaN,lncRNA


In [45]:
# No ref gene could be assigned but tx have exon overlap with GENCODE
combined_df[(combined_df["Gene type"].isna()) & (combined_df["category"] != "Fully unannotated transcripts without any overlapping exons")]["stripped_ref_id"].unique()

array(['ENST00000351839', 'ENST00000391947', 'ENST00000636394',
       'ENST00000334318', 'ENST00000418951', 'ENST00000617484',
       'ENST00000612355', 'ENST00000458258', 'ENST00000756245',
       'ENST00000797550'], dtype=object)

In [46]:
# Manual assignment
manual_gene_info = {
    "ENST00000351839": {"Gene stable ID": "ENSG00000165119", "Gene name": "HNRNPK", "Gene type": "protein_coding"},
    "ENST00000636394": {"Gene stable ID": "ENSG00000279170", "Gene name": "TSTD3", "Gene type": "protein_coding"},
    "ENST00000617484": {"Gene stable ID": "ENSG00000109819", "Gene name": "PPARGC1A", "Gene type": "protein_coding"},
    "ENST00000756245": {"Gene stable ID": "ENSG00000298529", "Gene name": "ENSG00000298529", "Gene type": "lncRNA"},

    "ENST00000391947": {"Gene stable ID": "ENSG00000198625", "Gene name": "MDM4", "Gene type": "protein_coding"},
    "ENST00000334318": {"Gene stable ID": "ENSG00000119314", "Gene name": "PTBP3", "Gene type": "protein_coding"},
    "ENST00000612355": {"Gene stable ID": "ENSG00000109819", "Gene name": "PPARGC1A", "Gene type": "protein_coding"},
    "ENST00000797550": {"Gene stable ID": "ENSG00000303858", "Gene name": "ENSG00000303858", "Gene type": "lncRNA"},

    "ENST00000418951": {"Gene stable ID": "ENSG00000099968", "Gene name": "BCL2L13", "Gene type": "protein_coding"},
    "ENST00000458258": {"Gene stable ID": "ENSG00000119314", "Gene name": "PTBP3", "Gene type": "protein_coding"} 
}

for ref_id, gene_info in manual_gene_info.items():
    mask = combined_df["stripped_ref_id"] == ref_id
    for col, value in gene_info.items():
        combined_df.loc[mask, col] = value

In [47]:
combined_df[(combined_df["Gene type"].isna()) & (combined_df["category"] != "Fully unannotated transcripts with non-overlapping exons")]["stripped_ref_id"].unique()

array(['None'], dtype=object)

In [48]:
gene_type_map = {
    'protein_coding': 'protein_coding',
    'lncRNA': 'lncRNA'
}

# Apply the mapping to make a new category column
combined_df['gene_category'] = combined_df['Gene type'].map(gene_type_map)

# Fill the rest: NaN --> 'unmapped', others --> 'other_noncoding'
combined_df['gene_category'] = combined_df['gene_category'].fillna(
    combined_df['Gene type'].apply(lambda x: 'unmapped' if pd.isna(x) else 'other_noncoding')
)
combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,NaN,lncRNA,lncRNA


In [49]:
perfect_matches = combined_df[combined_df['category'] == "GENCODE-annotated transcripts"]

In [50]:
# Find cases where multiple transcripts have same GENCODE perfect match (redundancy)
perfect_matches[(perfect_matches['ref_id'].duplicated(keep=False))].sort_values("ref_id")

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category
11512,ENST00000204566.2,chr15,"[(64963021, 64963736), (64965320, 64965460), (...","[(64963022, 64963736), (64965320, 64965460), (...",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000204566.7,ENST00000204566,ENSG00000090487,SPG21,protein_coding,protein_coding
12724,MICT00000118180.1,chr15,"[(64963022, 64963736), (64965320, 64965460), (...","[(64963022, 64963736), (64965320, 64965460), (...",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000204566.7,ENST00000204566,ENSG00000090487,SPG21,protein_coding,protein_coding


In [51]:
# Drop the transcript with worse match to GENCODE ref
combined_df = combined_df[combined_df['transcript_id'] != 'ENST00000204566.2']

In [52]:
# Fill NA gene names with the gene ID
combined_df['Gene name'] = combined_df['Gene name'].fillna(combined_df['Gene stable ID'])

## Add transcript expression info

In [53]:
tpm_df = pd.read_csv( '../data_processed/TSS_info_w_lift.tsv', sep='\t')
tpm_df.head()

,chromosome,tx_start,tx_end,strand,transcript_id,gene_id,gene_name,Gene class,gene_type,transcript_type,...,low_tpm,peak,"TSS expression high glucose (Salmon, TPM)",Relative TSS usage,TSS_type,distance_to_closest_TSS,Distance to closest Gencode TSS,name,new_1st_exon,new_name
0,chr1,14362,29370,-,ENST00000423562.1,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,...,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1
1,chr1,14359,29350,-,PBT00037199,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,...,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1
2,chr1,14359,29350,-,PBT00037200,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,...,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1
3,chr1,14360,29364,-,PBT00037201,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,...,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1
4,chr1,14362,29346,-,PBT00037202,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,...,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1


In [54]:
pd.set_option('display.max_columns', None)

In [55]:
# Add peak_name to combined_df
combined_df = combined_df.merge(
    tpm_df[['transcript_id', 'new_name', 'peak']].rename(columns={'transcript_id': 'merge_key', 'new_name': 'peak_name', 'peak': 'peak_type'}),
    how='left',
    left_on='transcript_id',
    right_on='merge_key'
).drop(columns='merge_key')

In [56]:
# Assign new Gene ID/name to unmapped tx
mask = combined_df['Gene stable ID'].isna()

peak_ids = combined_df.loc[mask, 'peak_name'].str.extract(r'(Peak_\d+)', expand=False)
combined_df.loc[mask, 'Gene stable ID'] = peak_ids
combined_df.loc[mask, 'Gene name'] = peak_ids

In [57]:
combined_df[combined_df["Gene name"].isna()]

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type


In [58]:
# Add back the TPMs per transcript 
tpm_df['mean_TPM_ref'] = tpm_df['relative_tx_exp'] * tpm_df['gene_exp_ref'] 

In [59]:
# Remove rows with no expression and log transform
tpm_df = tpm_df[tpm_df['mean_TPM_ref'] != 0].copy()
tpm_df['log2_tx_exp'] = np.log2(tpm_df['mean_TPM_ref'])
tpm_df.head()

,chromosome,tx_start,tx_end,strand,transcript_id,gene_id,gene_name,Gene class,gene_type,transcript_type,protein_id_orf_summary,transcript_classification,Transcript expression high glucose (TPM),gene_exp_ref,relative_tx_exp,CAGE_ID,TSS_start,TSS_end,high_tpm,low_tpm,peak,"TSS expression high glucose (Salmon, TPM)",Relative TSS usage,TSS_type,distance_to_closest_TSS,Distance to closest Gencode TSS,name,new_1st_exon,new_name,mean_TPM_ref,log2_tx_exp
0,chr1,14362,29370,-,ENST00000423562.1,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,UniProtKB=B3KS13,Known Protein coding,0.219906,27.348529,0.008041,chr1:29336-29359:-,29336,29359,17.58083,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1,0.219906,-2.185038
1,chr1,14359,29350,-,PBT00037199,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,NaN,Novel Protein coding not overlapping annotated...,3.422413,27.348529,0.125141,chr1:29336-29359:-,29336,29359,17.58083,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1,3.422413,1.775014
2,chr1,14359,29350,-,PBT00037200,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,"Closest_UniProtKB=A9QQ17, percentage_UniProtKB...",Novel Protein coding not overlapping annotated...,0.172042,27.348529,0.006291,chr1:29336-29359:-,29336,29359,17.58083,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1,0.172042,-2.539164
3,chr1,14360,29364,-,PBT00037201,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,NaN,Non coding,0.071035,27.348529,0.002597,chr1:29336-29359:-,29336,29359,17.58083,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1,0.071035,-3.815321
4,chr1,14362,29346,-,PBT00037202,ENSG00000227232,WASH7P,misc RNA,pseudogene,new_transcript,NaN,Non coding,0.322260,27.348529,0.011783,chr1:29336-29359:-,29336,29359,17.58083,17.856425,Robust,27.348529,1.0,unique TSS,211,>100bp,chr1:29336-29359_Peak_1,yes,chr1:29336-29359_Peak_1,0.322260,-1.633703


In [60]:
# Merge tx tpm info with comparison results
combined_df = combined_df.merge(
    tpm_df[['transcript_id', 'mean_TPM_ref', 'log2_tx_exp']].rename(columns={'transcript_id': 'merge_key'}),
    how='left',
    left_on='transcript_id',
    right_on='merge_key'
).drop(columns='merge_key')

In [61]:
len(combined_df)

376342

In [62]:
# Add the gene expression (sum of all the transcripts from the same gene) 
f = lambda x: x.sum()
combined_df['gene_exp_ref'] = combined_df.groupby('Gene stable ID')['mean_TPM_ref'].transform(f)

In [63]:
# get the transcript relative expression 
combined_df['relative_tx_exp'] = combined_df['mean_TPM_ref'] / combined_df['gene_exp_ref']

In [64]:
# add the sum of TPM expression of all the transcripts from one gene starting from the same TSS
f = lambda x: x.sum()
combined_df['TSS_TPM_per_gene'] = combined_df.groupby(['Gene stable ID','peak_name'])['mean_TPM_ref'].transform(f)

In [65]:
combined_df['Relative TSS usage'] = combined_df['TSS_TPM_per_gene']/combined_df['gene_exp_ref']

In [66]:
# control that no gene has an total relative expression above 1
combined_df[(combined_df['Relative TSS usage'] > 1)]

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage


In [67]:
combined_df["TSS_rank"] = combined_df.groupby("Gene stable ID")["Relative TSS usage"].rank(method="dense", ascending=False)

def get_TSS_type(row):
    if row["Relative TSS usage"] == 1:
        return "unique TSS"
    elif row["TSS_rank"] == 1:
        return "main TSS"
    elif row["TSS_rank"] == 2:
        return "secondary TSS"
    else:
        return "minor TSS"

combined_df["TSS_type"] = combined_df.apply(get_TSS_type, axis=1)
combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_rank,TSS_type
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding,chr1:61077221-61077380_Peak_3111,Permissive,0.178215,-2.488310,4.820350,0.036971,0.213765,0.044346,4.0,minor TSS
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA,chr1:160261726-160261763_Peak_5604,Permissive,0.358414,-1.480300,0.743874,0.481821,0.743874,1.000000,1.0,unique TSS
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding,chr1:239386576-239386587_Peak_7900,Robust,0.468766,-1.093061,3.127170,0.149901,1.421490,0.454561,1.0,main TSS
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding,chr1:23217498-23217499_Peak_1212,Permissive,0.064644,-3.951339,0.116743,0.553728,0.116743,1.000000,1.0,unique TSS
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,ENSG00000300424,lncRNA,lncRNA,chr1:233327323-233327461_Peak_7654,Permissive,0.616169,-0.698603,2.291033,0.268948,2.093885,0.913948,1.0,main TSS


In [68]:
combined_df = combined_df.drop(columns=["TSS_rank"])

## Finding unannotated TSSs

In [69]:
# Extract peak start and end positions
combined_df[["TSS_start", "TSS_end"]] = (combined_df["peak_name"].str.extract(r":(\d+)-(\d+)", expand=True)).astype("Int64")

combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,TSS_start,TSS_end
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding,chr1:61077221-61077380_Peak_3111,Permissive,0.178215,-2.488310,4.820350,0.036971,0.213765,0.044346,minor TSS,61077221,61077380
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA,chr1:160261726-160261763_Peak_5604,Permissive,0.358414,-1.480300,0.743874,0.481821,0.743874,1.000000,unique TSS,160261726,160261763
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding,chr1:239386576-239386587_Peak_7900,Robust,0.468766,-1.093061,3.127170,0.149901,1.421490,0.454561,main TSS,239386576,239386587
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding,chr1:23217498-23217499_Peak_1212,Permissive,0.064644,-3.951339,0.116743,0.553728,0.116743,1.000000,unique TSS,23217498,23217499
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,ENSG00000300424,lncRNA,lncRNA,chr1:233327323-233327461_Peak_7654,Permissive,0.616169,-0.698603,2.291033,0.268948,2.093885,0.913948,main TSS,233327323,233327461


In [70]:
# Load gencode v47 transcripts
gencode_v47_transcripts = pd.read_csv("../data_raw/gencode.v47.transcript.tsv", sep="\t", dtype=str)
gencode_v47_transcripts["start"] = gencode_v47_transcripts["start"].astype(int)
gencode_v47_transcripts["end"] = gencode_v47_transcripts["end"].astype(int)

gencode_v47_transcripts = gencode_v47_transcripts.rename(columns={"seqname": "chr"})

gencode_v47_transcripts.head()

,chr,source,feature,start,end,score,strand,frame,gene_id,transcript_id,gene_type,gene_name,transcript_type,transcript_name,level,tag,transcript_support_level,havana_transcript,hgnc_id,ont,havana_gene,protein_id,ccdsid
0,chr1,HAVANA,transcript,11121,14413,.,+,.,ENSG00000290825.2,ENST00000832824.1,lncRNA,DDX11L16,lncRNA,DDX11L16-260,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11125,14405,.,+,.,ENSG00000290825.2,ENST00000832825.1,lncRNA,DDX11L16,lncRNA,DDX11L16-261,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,transcript,11410,14413,.,+,.,ENSG00000290825.2,ENST00000832826.1,lncRNA,DDX11L16,lncRNA,DDX11L16-262,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,transcript,11411,14413,.,+,.,ENSG00000290825.2,ENST00000832827.1,lncRNA,DDX11L16,lncRNA,DDX11L16-263,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,transcript,11426,14409,.,+,.,ENSG00000290825.2,ENST00000832828.1,lncRNA,DDX11L16,lncRNA,DDX11L16-264,2,TAGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# For each TSS find the closest GENCODE tx start
transcript_groups = {
    (chr, strand): df for (chr, strand), df in gencode_v47_transcripts.groupby(["chr", "strand"])
}

def compute_closest_distance_and_transcript(row):
    key = (row["chr"], row["Strand"])
    matches = transcript_groups.get(key, None)

    if matches is None or matches.empty or pd.isna(row["TSS_start"]) or pd.isna(row["TSS_end"]):
        return pd.Series([pd.NA, pd.NA])

    tss_midpoint = (row["TSS_start"] + row["TSS_end"]) // 2
    
    if row["Strand"] == "+":
        distances = np.minimum(np.minimum((matches["start"] - row["TSS_start"]).abs(), (matches["start"] - row["TSS_end"]).abs()), (matches["start"] - tss_midpoint).abs())
    elif row["Strand"] == "-":
        distances = np.minimum(np.minimum((matches["end"] - row["TSS_start"]).abs(), (matches["end"] - row["TSS_end"]).abs()), (matches["end"] - tss_midpoint).abs())
    else:
        return pd.Series([pd.NA, pd.NA])

    min_idx = distances.idxmin()
    min_distance = distances.loc[min_idx]
    closest_transcript = matches.loc[min_idx, "transcript_id"]

    return pd.Series([min_distance, closest_transcript])


tqdm.pandas()

combined_df[["distance_to_closest_TSS", "closest_TSS_ref_id"]] = combined_df.progress_apply(compute_closest_distance_and_transcript, axis=1)
combined_df["distance_to_closest_TSS"] = combined_df["distance_to_closest_TSS"].astype("Int64")

100%|██████████| 376342/376342 [02:52<00:00, 2176.32it/s]


In [72]:
f = lambda x: "<=100bp" if x <= 100 else ">100bp"
combined_df['TSS distance category'] = combined_df['distance_to_closest_TSS'].apply(f)

In [73]:
# Create TSS name based on annotation status
combined_df["TSS_name"] = (
    "P" 
    + combined_df["peak_name"].str.extract(r"Peak_(\d+)", expand=False)
    + np.where(
        combined_df["TSS distance category"] == "<=100bp",
        "_A",
        "_U"
    )
)
combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,TSS_start,TSS_end,distance_to_closest_TSS,closest_TSS_ref_id,TSS distance category,TSS_name
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding,chr1:61077221-61077380_Peak_3111,Permissive,0.178215,-2.488310,4.820350,0.036971,0.213765,0.044346,minor TSS,61077221,61077380,6,ENST00000699964.1,<=100bp,P3111_A
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA,chr1:160261726-160261763_Peak_5604,Permissive,0.358414,-1.480300,0.743874,0.481821,0.743874,1.000000,unique TSS,160261726,160261763,0,ENST00000442130.1,<=100bp,P5604_A
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding,chr1:239386576-239386587_Peak_7900,Robust,0.468766,-1.093061,3.127170,0.149901,1.421490,0.454561,main TSS,239386576,239386587,8,ENST00000676153.1,<=100bp,P7900_A
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding,chr1:23217498-23217499_Peak_1212,Permissive,0.064644,-3.951339,0.116743,0.553728,0.116743,1.000000,unique TSS,23217498,23217499,3,ENST00000374619.2,<=100bp,P1212_A
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,ENSG00000300424,lncRNA,lncRNA,chr1:233327323-233327461_Peak_7654,Permissive,0.616169,-0.698603,2.291033,0.268948,2.093885,0.913948,main TSS,233327323,233327461,3,ENST00000771586.1,<=100bp,P7654_A


## Collapsing TSS groups

In [74]:
unannotated_genes = combined_df[combined_df["category"] == "Fully unannotated transcripts without any overlapping exons"].copy()

In [75]:
unannotated_genes

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,TSS_start,TSS_end,distance_to_closest_TSS,closest_TSS_ref_id,TSS distance category,TSS_name
336566,ENCT00000000219.1,chr1,"[(1162877, 1163719), (1163993, 1164042)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_111,Peak_111,NaN,unmapped,chr1:1162789-1162942_Peak_111,Robust,0.018685,-5.741955,0.339126,0.055098,0.339126,1.0,unique TSS,1162789,1162942,4162,ENST00000384997.3,>100bp,P111_U
336567,ENCT00000000276.1,chr1,"[(1349559, 1355076), (1355263, 1355740), (1357...",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_145,Peak_145,NaN,unmapped,chr1:1349517-1349586_Peak_145,Permissive,0.836916,-0.256846,0.867767,0.964448,0.867767,1.0,unique TSS,1349517,1349586,679,ENST00000806659.1,>100bp,P145_U
336568,ENCT00000002194.1,chr1,"[(17439849, 17449625)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_910,Peak_910,NaN,unmapped,chr1:17439844-17439937_Peak_910,Permissive,0.085408,-3.549480,0.085408,1.000000,0.085408,1.0,unique TSS,17439844,17439937,20004,ENST00000835437.1,>100bp,P910_U
336569,ENCT00000002302.1,chr1,"[(19485741, 19489397)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_996,Peak_996,NaN,unmapped,chr1:19485730-19485780_Peak_996,Permissive,0.114061,-3.132117,0.168185,0.678191,0.168185,1.0,unique TSS,19485730,19485780,1327,ENST00000648702.1,>100bp,P996_U
336570,ENCT00000002410.1,chr1,"[(20508236, 20510041)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_1039,Peak_1039,NaN,unmapped,chr1:20508218-20508233_Peak_1039,Permissive,0.059096,-4.080791,0.059096,1.000000,0.059096,1.0,unique TSS,20508218,20508233,17756,ENST00000443195.1,>100bp,P1039_U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339852,STRT02284050,chrX,"[(74034396, 74034874)]",NaN,Fully unannotated transcripts without any over...,NaN,-,Unannotated transcripts without any annotated ...,None,None,Peak_79446,Peak_79446,NaN,unmapped,chrX:74034838-74034839_Peak_79446,Permissive,1.715846,0.778920,1.715846,1.000000,1.715846,1.0,unique TSS,74034838,74034839,21893,ENST00000638437.1,>100bp,P79446_U
339853,STRT02289282,chrX,"[(51805614, 51805945)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_79008,Peak_79008,NaN,unmapped,chrX:51805624-51805625_Peak_79008,Permissive,0.318309,-1.651502,0.318309,1.000000,0.318309,1.0,unique TSS,51805624,51805625,2617,ENST00000375772.7,>100bp,P79008_U
339854,STRT02289408,chrX,"[(57617393, 57618659)]",NaN,Fully unannotated transcripts without any over...,NaN,+,Unannotated transcripts without any annotated ...,None,None,Peak_79202,Peak_79202,NaN,unmapped,chrX:57617355-57617356_Peak_79202,Permissive,0.129846,-2.945122,0.129846,1.000000,0.129846,1.0,unique TSS,57617355,57617356,23432,ENST00000818758.1,>100bp,P79202_U
339855,STRT02290336,chrX,"[(19990302, 19990805)]",NaN,Fully unannotated transcripts without any over...,NaN,-,Unannotated transcripts without any annotated ...,None,None,Peak_78445,Peak_78445,NaN,unmapped,chrX:19990802-19990828_Peak_78445,Permissive,0.101023,-3.307240,0.260752,0.387430,0.260752,1.0,unique TSS,19990802,19990828,233,ENST00000379682.9,>100bp,P78445_U


In [76]:
def get_tx_coords(exon_list):
    if not exon_list or not isinstance(exon_list, (list, tuple)):
        return (None, None)
    starts = [e[0] for e in exon_list]
    ends = [e[1] for e in exon_list]
    return (min(starts), max(ends))

In [77]:
unannotated_genes[["tx_start", "tx_end"]] = unannotated_genes["exons"].apply(
    lambda x: pd.Series(get_tx_coords(x))
)

In [78]:
transcript_coords = unannotated_genes[["chr", "Strand", "Gene name", "transcript_id", "tx_start", "tx_end"]]
transcript_coords

,chr,Strand,Gene name,transcript_id,tx_start,tx_end
336566,chr1,+,Peak_111,ENCT00000000219.1,1162877,1164042
336567,chr1,+,Peak_145,ENCT00000000276.1,1349559,1358279
336568,chr1,+,Peak_910,ENCT00000002194.1,17439849,17449625
336569,chr1,+,Peak_996,ENCT00000002302.1,19485741,19489397
336570,chr1,+,Peak_1039,ENCT00000002410.1,20508236,20510041
...,...,...,...,...,...,...
339852,chrX,-,Peak_79446,STRT02284050,74034396,74034874
339853,chrX,+,Peak_79008,STRT02289282,51805614,51805945
339854,chrX,+,Peak_79202,STRT02289408,57617393,57618659
339855,chrX,-,Peak_78445,STRT02290336,19990302,19990805


In [79]:
transcript_coords["Gene name"].nunique()

1704

In [80]:
gene_coords = (
    transcript_coords
    .groupby(["chr", "Strand", "Gene name"], as_index=False)
    .agg(min_start=("tx_start", "min"), max_end=("tx_end", "max"))
)
gene_coords = gene_coords.rename(columns={"Gene name": "Gene_name"})
gene_coords

,chr,Strand,Gene_name,min_start,max_end
0,chr1,+,Peak_1039,20508236,20510041
1,chr1,+,Peak_111,1162877,1166499
2,chr1,+,Peak_1187,22342372,22353088
3,chr1,+,Peak_1234,23555118,23556240
4,chr1,+,Peak_1238,23559643,23560425
...,...,...,...,...,...
1699,chrX,-,Peak_80211,126472363,126472699
1700,chrX,-,Peak_80214,129382617,129382957
1701,chrX,-,Peak_80261,130401109,130401897
1702,chrX,-,Peak_80530,153927441,153928228


In [81]:
# Sort first
gene_coords = gene_coords.sort_values(["chr", "Strand", "min_start"]).reset_index(drop=True)

# Compute running maximum end per chromosome/strand
gene_coords["running_max_end"] = (
    gene_coords.groupby(["chr", "Strand"])["max_end"]
    .cummax()
)

# A new group starts when the current start > previous running max_end
gene_coords["prev_running_max_end"] = gene_coords.groupby(["chr", "Strand"])["running_max_end"].shift()
gene_coords["new_group"] = (
    (gene_coords["chr"] != gene_coords["chr"].shift()) |
    (gene_coords["Strand"] != gene_coords["Strand"].shift()) |
    (gene_coords["min_start"] > gene_coords["prev_running_max_end"])
).astype(int)

gene_coords["group_id"] = gene_coords["new_group"].cumsum()

# Summarize groups
groups = (
    gene_coords.groupby(["chr", "Strand", "group_id"])
    .agg(
        group_start=("min_start", "min"),
        group_end=("max_end", "max"),
        Genes=("Gene_name", list),
        group_size=("Gene_name", "count")
    )
    .reset_index()
)
groups

,chr,Strand,group_id,group_start,group_end,Genes,group_size
0,chr1,+,1,1162877,1166499,[Peak_111],1
1,chr1,+,2,1349550,1358279,[Peak_145],1
2,chr1,+,3,1375573,1378310,[Peak_150],1
3,chr1,+,4,2189680,2191740,[Peak_264],1
4,chr1,+,5,2199524,2202776,[Peak_274],1
...,...,...,...,...,...,...,...
1603,chrX,-,1604,126472363,126472699,[Peak_80211],1
1604,chrX,-,1605,129382617,129382957,[Peak_80214],1
1605,chrX,-,1606,130401109,130401897,[Peak_80261],1
1606,chrX,-,1607,153927441,153928228,[Peak_80530],1


In [82]:
def make_combined_name(genes):
    if len(genes) == 1:
        return genes[0]
    else:
        # Extract numeric parts and sort
        nums = sorted([int(g.split("_")[1]) for g in genes])
        return f"Peak_{nums[0]}-{nums[-1]}"

# Apply to create new column
groups["new_gene_name"] = groups["Genes"].apply(make_combined_name)

groups.sort_values(by="group_size", ascending=False)

,chr,Strand,group_id,group_start,group_end,Genes,group_size,new_gene_name
1462,chr8,+,1463,117244748,117311898,"[Peak_74199, Peak_74200, Peak_74201, Peak_7420...",5,Peak_74199-74203
566,chr16,-,567,57564434,57607224,"[Peak_29184, Peak_29170, Peak_29174, Peak_2917...",5,Peak_29170-29184
1008,chr3,+,1009,122184040,122253386,"[Peak_54073, Peak_54086, Peak_54100, Peak_54102]",4,Peak_54073-54102
181,chr10,+,182,97772165,97842556,"[Peak_10465, Peak_10467, Peak_10468, Peak_10469]",4,Peak_10465-10469
571,chr16,-,572,85445590,85465053,"[Peak_30027, Peak_30030, Peak_30031]",3,Peak_30027-30031
...,...,...,...,...,...,...,...,...
543,chr16,+,544,75266030,75269181,[Peak_29846],1,Peak_29846
542,chr16,+,543,72221946,72228455,[Peak_29730],1,Peak_29730
541,chr16,+,542,71809311,71813524,[Peak_29702],1,Peak_29702
540,chr16,+,541,70946388,70948148,[Peak_29657],1,Peak_29657


In [83]:
# Create gene name mapping
gene_to_new = {}
for _, row in groups.iterrows():
    for gene in row["Genes"]:
        gene_to_new[gene] = row["new_gene_name"]

In [84]:
# Replace gene names
mask = combined_df["Gene name"].str.startswith("Peak")
combined_df.loc[mask, "Gene name"] = combined_df.loc[mask, "Gene name"].map(gene_to_new)
combined_df.loc[mask, "Gene stable ID"] = combined_df.loc[mask, "Gene stable ID"].map(gene_to_new)

In [87]:
combined_df["Gene stable ID"].nunique()

22015

In [91]:
combined_df

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,TSS_start,TSS_end,distance_to_closest_TSS,closest_TSS_ref_id,TSS distance category,TSS_name
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding,chr1:61077221-61077380_Peak_3111,Permissive,0.178215,-2.488310,4.820350,0.036971,0.213765,0.044346,minor TSS,61077221,61077380,6,ENST00000699964.1,<=100bp,P3111_A
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA,chr1:160261726-160261763_Peak_5604,Permissive,0.358414,-1.480300,0.743874,0.481821,0.743874,1.000000,unique TSS,160261726,160261763,0,ENST00000442130.1,<=100bp,P5604_A
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding,chr1:239386576-239386587_Peak_7900,Robust,0.468766,-1.093061,3.127170,0.149901,1.421490,0.454561,main TSS,239386576,239386587,8,ENST00000676153.1,<=100bp,P7900_A
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding,chr1:23217498-23217499_Peak_1212,Permissive,0.064644,-3.951339,0.116743,0.553728,0.116743,1.000000,unique TSS,23217498,23217499,3,ENST00000374619.2,<=100bp,P1212_A
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,ENSG00000300424,lncRNA,lncRNA,chr1:233327323-233327461_Peak_7654,Permissive,0.616169,-0.698603,2.291033,0.268948,2.093885,0.913948,main TSS,233327323,233327461,3,ENST00000771586.1,<=100bp,P7654_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376337,STRT02294272,chrY,"[(12662355, 12662706), (12664640, 12664686), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,+,Unannotated transcripts with all exons annotated,ENST00000651177.1,ENST00000651177,ENSG00000114374,USP9Y,protein_coding,protein_coding,chrY:12662270-12662588_Peak_80721,Robust,0.035458,-4.817728,8.744087,0.004055,8.744087,1.000000,unique TSS,12662270,12662588,51,ENST00000417071.1,<=100bp,P80721_A
376338,STRT02294670,chrY,"[(20575798, 20575887), (20582590, 20582693), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,+,Unannotated transcripts with all exons annotated,ENST00000382772.3,ENST00000382772,ENSG00000198692,EIF1AY,protein_coding,protein_coding,chrY:20575721-20575776_Peak_80778,Robust,3.717255,1.894237,20.208525,0.183945,20.208525,1.000000,unique TSS,20575721,20575776,0,ENST00000361365.7,<=100bp,P80778_A
376339,STRT02294844,chrY,"[(12662506, 12662706), (12664640, 12664686), (...",NaN,Unannotated transcripts composed entirely of a...,NaN,+,Unannotated transcripts with all exons annotated,ENST00000651177.1,ENST00000651177,ENSG00000114374,USP9Y,protein_coding,protein_coding,chrY:12662270-12662588_Peak_80721,Robust,0.248549,-2.008399,8.744087,0.028425,8.744087,1.000000,unique TSS,12662270,12662588,51,ENST00000417071.1,<=100bp,P80721_A
376340,STRT022948

In [92]:
combined_df.to_csv('../data_processed/curated_HIT.tsv', sep='\t', index=False)

## Renaming transcript IDs

In [93]:
tx_source_map = HIT_hg38_df[['source', 'transcript_id']].drop_duplicates()
combined_df = combined_df.merge(tx_source_map, on='transcript_id', how='left')
combined_df.head()

,transcript_id,chr,exons,matched_to,category,unannotated_exons,Strand,summarized_category,ref_id,stripped_ref_id,Gene stable ID,Gene name,Gene type,gene_category,peak_name,peak_type,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,TSS_start,TSS_end,distance_to_closest_TSS,closest_TSS_ref_id,TSS distance category,TSS_name,source
0,ENCT00000006334.1,chr1,"[(61077274, 61077628), (61077916, 61079203)]","[(61077326, 61077628), (61077916, 61079231)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000699992.1,ENST00000699992,ENSG00000162599,NFIA,protein_coding,protein_coding,chr1:61077221-61077380_Peak_3111,Permissive,0.178215,-2.488310,4.820350,0.036971,0.213765,0.044346,minor TSS,61077221,61077380,6,ENST00000699964.1,<=100bp,P3111_A,FANTOM_cat
1,ENCT00000013056.1,chr1,"[(160261734, 160261922), (160281430, 160281892)]","[(160261682, 160261922), (160281430, 160281935)]",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000756371.1,ENST00000756371,ENSG00000228606,DCAF8-DT,lncRNA,lncRNA,chr1:160261726-160261763_Peak_5604,Permissive,0.358414,-1.480300,0.743874,0.481821,0.743874,1.000000,unique TSS,160261726,160261763,0,ENST00000442130.1,<=100bp,P5604_A,FANTOM_cat
2,ENCT00000019620.1,chr1,"[(239386568, 239387227), (239492709, 239492807...","[(239386568, 239387227), (239492709, 239492807...",GENCODE-annotated transcripts,NaN,+,GENCODE-annotated transcripts,ENST00000676153.1,ENST00000676153,ENSG00000133019,CHRM3,protein_coding,protein_coding,chr1:239386576-239386587_Peak_7900,Robust,0.468766,-1.093061,3.127170,0.149901,1.421490,0.454561,main TSS,239386576,239386587,8,ENST00000676153.1,<=100bp,P7900_A,FANTOM_cat
3,ENCT00000022754.1,chr1,"[(23191893, 23195001), (23217291, 23217499)]","[(23191895, 23195001), (23217291, 23217502)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000374619.2,ENST00000374619,ENSG00000179546,HTR1D,protein_coding,protein_coding,chr1:23217498-23217499_Peak_1212,Permissive,0.064644,-3.951339,0.116743,0.553728,0.116743,1.000000,unique TSS,23217498,23217499,3,ENST00000374619.2,<=100bp,P1212_A,FANTOM_cat
4,ENCT00000039204.1,chr1,"[(233319835, 233321155), (233327229, 233327357)]","[(233319834, 233321155), (233327229, 233327455)]",GENCODE-annotated transcripts,NaN,-,GENCODE-annotated transcripts,ENST00000771587.1,ENST00000771587,ENSG00000300424,ENSG00000300424,lncRNA,lncRNA,chr1:233327323-233327461_Peak_7654,Permissive,0.616169,-0.698603,2.291033,0.268948,2.093885,0.913948,main TSS,233327323,233327461,3,ENST00000771586.1,<=100bp,P7654_A,FANTOM_cat


In [5]:
prefix_map = {
    'FANTOM_cat': 'FTMT',
    'PacBio': 'PBT',
    'StringTie': 'STRT',
    'ENSEMBL': 'NCBIT',
}

combined_df["new_tx_id"] = None

# assign GENCODE-annotated transcripts
mask = combined_df["category"] == "GENCODE-annotated transcripts"
combined_df.loc[mask, "new_tx_id"] = combined_df.loc[mask, "matched_ref_id"]

# generate IDs based on source
remaining = combined_df.loc[~mask].copy()

for source_name, prefix in prefix_map.items():
    source_mask = remaining["source"] == source_name
    idx = remaining[source_mask].index
    for i, row_idx in enumerate(idx, start=1):
        combined_df.at[row_idx, "new_tx_id"] = f"{prefix}{str(i).zfill(7)}"

In [6]:
combined_df[combined_df["new_tx_id"].str.startswith("NCBIT")]

,transcript_id,chr,exons,matched_to,matched_ref_id,category,unannotated_exons,best_match_ref_id,largest_overlap_gencode_id,Strand,...,peak_name,mean_TPM_ref,log2_tx_exp,gene_exp_ref,relative_tx_exp,TSS_TPM_per_gene,Relative TSS usage,TSS_type,source,new_tx_id
38927,ENST00000235329.5,chr1,"[(11980197, 11980484), (11981970, 11982114), (...","[(11980209, 11980484), (11981970, 11982114), (...",ENST00000675817.1,Unannotated transcripts fully contained within...,NaN,NaN,NaN,+,...,chr1:11980204-11980451_Peak_666,3.586003,1.842377,15.968124,0.224573,13.568596,0.849730,main TSS,ENSEMBL,NCBIT0000001
38928,ENST00000236914.3,chr1,"[(200027634, 200027911), (200043774, 200043892...","[(200027710, 200027911), (200039658, 200039795...",ENST00000367362.8,Unannotated transcripts fully contained within...,NaN,NaN,NaN,+,...,chr1:200027537-200027864_Peak_6480,4.262182,2.091592,21.202160,0.201026,12.154987,0.573290,main TSS,ENSEMBL,NCBIT0000002
38929,ENST00000239457.5,chr1,"[(173824659, 173825356), (173826687, 173826786...","[(173824673, 173825356), (173826687, 173826786...",ENST00000649689.2,Unannotated transcripts fully contained within...,NaN,NaN,NaN,+,...,chr1:173824474-173824687_Peak_6035,0.288137,-1.795171,6.000629,0.048018,5.583196,0.930435,main TSS,ENSEMBL,NCBIT0000003
38930,ENST00000253251.8,chr1,"[(10032959, 10033694), (10072028, 10072214), (...","[(10032958, 10033694), (10072028, 10072214), (...",ENST00000343090.11,Unannotated transcripts fully contained within...,NaN,NaN,NaN,+,...,chr1:10032954-10033166_Peak_545,5.618578,2.490205,10.907193,0.515126,10.885175,0.997981,main TSS,ENSEMBL,NCBIT0000004
38931,ENST00000261443.5,chr1,"[(114716920, 114718216), (114718613, 114718745...","[(114716913, 114718216), (114718613, 114718745...",ENST00000339438.10,Unannotated transcripts fully contained within...,NaN,NaN,NaN,-,...,chr1:114757960-114757994_Peak_4265,3.338205,1.739073,193.694624,0.017234,137.989329,0.712407,main TSS,ENSEMBL,NCBIT0000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375357,ENST00000543642.1,chrX,"[(37349330, 37349469), (37406209, 37406259), (...",NaN,"['ENST00000491253.1', 'ENST00000470290.1', 'EN...",Unannotated transcripts composed entirely of a...,NaN,ENST00000378628.9,NaN,+,...,chrX:37349326-37349369_Peak_78599,0.504692,-0.986525,2.708327,0.186348,2.708327,1.000000,unique TSS,ENSEMBL,NCBIT0014842
375358,ENST00000545566.1,chrX,"[(13653123, 13653188), (13655289, 13655373), (...",NaN,"['ENST00000696126.2', 'ENST00000380600.2', 'EN...",Unannotated transcripts composed entirely of a...,NaN,ENST00000696128.1,NaN,+,...,chrX:13653136-13653193_Peak_78335,0.097810,-3.353878,1.731590,0.056486,1.731590,1.000000,unique TSS,ENSEMBL,NCBIT0014843
375359,ENST00000545618.1,chrX,"[(64916370, 64917896), (64919042, 64919204), (...",NaN,"['ENST00000374839.8', 'ENST00000488831.5', 'EN...",Unannotated transcripts composed entirely of a...,NaN,ENST00000703136.1,NaN,-,...,chrX:64976443-64976455_Peak_79235,0.071311,-3.809740,3.810785,0.018713,2.025059,0.531402,main TSS,ENSEMBL,NCBIT0014844
375360,ENST00000602791.1,chrX,"[(77910631, 77910835), (77962623, 77964890)]",NaN,"['ENST00000341514.11', 'ENST00000602791.2', 'E...",Unannotated transcripts composed entirely of a...,NaN,ENST00000602791.2,NaN,+,...,chrX:77910639-77910745_Peak_79619,0.409518,-1.288001,0.586949,0.697707,0.586949,1.000000,unique TSS,ENSEMBL,NCBIT0014845


In [ ]:
#combined_df.to_csv('HIT_w_new_tx_ID.tsv', sep='\t', index=False)